In [ ]:
import os
import xml.etree.ElementTree as ET
from multiprocessing import Pool

In [ ]:
def create_pascal_voc(xml_path, image_filename, width, height, objects):
    annotation = ET.Element("annotation")
    ET.SubElement(annotation, "folder").text = "images"
    ET.SubElement(annotation, "filename").text = image_filename
    size = ET.SubElement(annotation, "size")
    ET.SubElement(size, "width").text = str(width)
    ET.SubElement(size, "height").text = str(height)
    ET.SubElement(size, "depth").text = "3"
    
    for obj in objects:
        obj_element = ET.SubElement(annotation, "object")
        ET.SubElement(obj_element, "name").text = obj[4]  # Class name
        ET.SubElement(obj_element, "pose").text = "Unspecified"
        ET.SubElement(obj_element, "truncated").text = "0"
        ET.SubElement(obj_element, "difficult").text = "0"
        bndbox = ET.SubElement(obj_element, "bndbox")
        ET.SubElement(bndbox, "xmin").text = str(obj[0])
        ET.SubElement(bndbox, "ymin").text = str(obj[1])
        ET.SubElement(bndbox, "xmax").text = str(obj[2])
        ET.SubElement(bndbox, "ymax").text = str(obj[3])
    
    tree = ET.ElementTree(annotation)
    tree.write(xml_path)

def process_label_file(args):
    label_path, annotation_dir, image_width, image_height, label_map = args
    filename = os.path.basename(label_path)
    xml_path = os.path.join(annotation_dir, filename.replace(".txt", ".xml"))
    image_filename = filename.replace(".txt", ".jpg")
    
    with open(label_path, "r") as file:
        lines = file.readlines()
        if not lines:
            print(f"Skipping empty label file: {label_path}")
            return 0
        
        objects = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 5:  # Assuming format: class x_center y_center width height (YOLO format)
                class_id, x_center, y_center, width, height = map(float, parts)
                class_name = label_map[str(int(class_id))]
                
                xmin = max(0, int((x_center - width / 2) * image_width))
                ymin = max(0, int((y_center - height / 2) * image_height))
                xmax = min(image_width, int((x_center + width / 2) * image_width))
                ymax = min(image_height, int((y_center + height / 2) * image_height))
                
                if (xmax - xmin) > 10 and (ymax - ymin) > 10:
                    objects.append((xmin, ymin, xmax, ymax, class_name))
        
        if objects:
            create_pascal_voc(xml_path, image_filename, image_width, image_height, objects)
            print(f"Created {xml_path}")
            return 1
    return 0

def process_labels(label_dirs, annotation_dirs):
    file_count = 0
    image_width, image_height = 4000, 3000
    label_map = {
        "0": "Hyphaene petersiana",
        "1": "Philenoptera violacea",
        "2": "Combretum imberbe"
    }
    
    tasks = []
    for label_dir, annotation_dir in zip(label_dirs, annotation_dirs):
        os.makedirs(annotation_dir, exist_ok=True)
        for filename in os.listdir(label_dir):
            if filename.endswith(".txt"):
                label_path = os.path.join(label_dir, filename)
                tasks.append((label_path, annotation_dir, image_width, image_height, label_map))
    
    with Pool() as pool:
        results = pool.map(process_label_file, tasks)
        file_count = sum(results)
    
    print(f"Processing complete. Total XML files created: {file_count}")

# Define label and annotation directories
label_dirs = [
    "datasets/mokolwane/train/labels", "datasets/mopororo/train/labels", "datasets/motswere/train/labels",
    "datasets/mokolwane/val/labels", "datasets/mopororo/val/labels", "datasets/motswere/val/labels"
]

annotation_dirs = [
    "datasets/mokolwane/train/annotations", "datasets/mopororo/train/annotations", "datasets/motswere/train/annotations",
    "datasets/mokolwane/val/annotations", "datasets/mopororo/val/annotations", "datasets/motswere/val/annotations"
]

# Convert labels to Pascal VOC
process_labels(label_dirs, annotation_dirs)